<div  style="display: table; width: 100%; color: #034ea2; background:#ffffff ; line-height: 100px;  text-align: center;" dir="rtl">
        <h2 style="display:table-cell;">جامعة القلمون  </h2>
        <h1 style="display:table-cell;"> <b>👨‍💻 اويس الحمود 👨‍💻</b></h1>
        <h2 style="display:table-cell;">201811330</h2>
    </div>

we need to import the following libraries.

In [87]:
import pandas as pd
import re
import snowballstemmer 
import arabicstopwords.arabicstopwords as stp
import os
import pyterrier as pt


removing Stop Words function

In [88]:
def remove_stopWords(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() : 
        if term not in stopWords :
            terms.append(term)
    return " ".join(terms)

a function to normalize the tweets

In [89]:
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

specify that we want to stem arabic text ,
define the stemming function


In [90]:
ar_stemmer = snowballstemmer.stemmer('arabic')
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])

define preProcessing func 

In [91]:
def preProcessing(text):
    return stem(normalize(remove_stopWords(text)))

init pyterrier

In [92]:
j_h=os.environ.get('JAVA_HOME',None)
if not j_h:
    os.environ['JAVA_HOME']='C:\\Program Files\\Java\\jdk-20\\bin'
if not pt.started():
    pt.init()

read data from file is mydata folder

In [93]:
i=0
docs_df = pd.DataFrame(columns=["docno", "raw_text"])
for filename in os.listdir("..\\mydata"):
    i+=1 
    filepath = os.path.join("..\\mydata", filename) 
    with open(filepath, "r") as file: 
        data = file.read()
    docs_df.loc[len(docs_df)]=[f'd{i}',data]
         
pd.set_option('display.max_colwidth',150)
docs_df.head()

,docno,raw_text
0,d1,هذه هي المحاضرة الثانية من مقرر نظم استرجاع المعلومات
1,d2,المحاضرة باللغة العربية لطلاب جامعة القلمون
2,d3,كان الجو غائماً جزئياً
3,d4,نأمل أن تفيد هذا المقرر طلاب جامعة القلومن
4,d5,هل أنتم سعداء بهذه التجربة


preProcessing for collection 

In [94]:
docs_df['text']=docs_df['raw_text'].apply(preProcessing)
docs_df.head()

,docno,raw_text,text
0,d1,هذه هي المحاضرة الثانية من مقرر نظم استرجاع المعلومات,محاضره ثانيه مقرر نظم استرجاع معلوم
1,d2,المحاضرة باللغة العربية لطلاب جامعة القلمون,محاضره لغه عربيه لطلاب جامع قلم
2,d3,كان الجو غائماً جزئياً,الجو غاءم جزءي
3,d4,نأمل أن تفيد هذا المقرر طلاب جامعة القلومن,نامل تفيد مقرر طلاب جامع قلوم
4,d5,هل أنتم سعداء بهذه التجربة,سعداء تجربه


In [95]:
indexer = pt.DFIndexer("..\\ll1",overwrite=True)
indexer.setProperty("tokeniser", "UTFTokeniser")
index_ref = indexer.index(docs_df["text"], docs_df["docno"])


C:\Users\new laptop\AppData\Local\Temp\ipykernel_17412\3834369858.py:3: DeprecationWarning: Setting of tokeniser property directly is deprecated
  index_ref = indexer.index(docs_df["text"], docs_df["docno"])


16:50:14.184 [main] ERROR org.terrier.structures.indexing.Indexer - Could not rename index
java.io.IOException: Rename of index structure file '..\ll1/data_1.direct.bf' (exists) to '..\ll1/data.direct.bf' (exists) failed - likely that source file is still open. Possible indexing bug?
	at org.terrier.structures.IndexUtil.renameIndex(IndexUtil.java:379)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:355)


In [96]:
print(index_ref.toString())
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

..\ll1/data.properties
Number of documents: 5
Number of terms: 20
Number of postings: 23
Number of fields: 0
Number of tokens: 23
Field names: []
Positions:   false



In [97]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString())) 

استرجاع -> term0 Nt=1 TF=1 maxTF=1 @{0 0 0} 
الجو -> term12 Nt=1 TF=1 maxTF=1 @{0 0 2} 
تجربه -> term19 Nt=1 TF=1 maxTF=1 @{0 0 6} 
تفيد -> term17 Nt=1 TF=1 maxTF=1 @{0 1 4} 
ثانيه -> term4 Nt=1 TF=1 maxTF=1 @{0 2 2} 
جامع -> term8 Nt=2 TF=2 maxTF=1 @{0 2 4} 
جزءي -> term11 Nt=1 TF=1 maxTF=1 @{0 3 4} 
سعداء -> term18 Nt=1 TF=1 maxTF=1 @{0 4 0} 
طلاب -> term15 Nt=1 TF=1 maxTF=1 @{0 4 6} 
عربيه -> term7 Nt=1 TF=1 maxTF=1 @{0 5 4} 
غاءم -> term13 Nt=1 TF=1 maxTF=1 @{0 6 0} 
قلم -> term9 Nt=1 TF=1 maxTF=1 @{0 6 4} 
قلوم -> term16 Nt=1 TF=1 maxTF=1 @{0 7 0} 
لطلاب -> term10 Nt=1 TF=1 maxTF=1 @{0 7 6} 
لغه -> term6 Nt=1 TF=1 maxTF=1 @{0 8 2} 
محاضره -> term3 Nt=2 TF=2 maxTF=1 @{0 8 6} 
معلوم -> term5 Nt=1 TF=1 maxTF=1 @{0 9 2} 
مقرر -> term2 Nt=2 TF=2 maxTF=1 @{0 9 4} 
نامل -> term14 Nt=1 TF=1 maxTF=1 @{0 10 2} 
نظم -> term1 Nt=1 TF=1 maxTF=1 @{0 11 0} 


In [105]:
model= pt.BatchRetrieve(index_ref, wmodel="TF_IDF")
docNoResult=model.search(' محاضره جامع')
docNoResult[['query','docno']]

,query,docno
0,محاضره جامع,d2
1,محاضره جامع,d1
2,محاضره جامع,d4


In [99]:
q='جامعة القلمون'
q=preProcessing(q)
print(q)
docNoResult= model.search(q)['docno'].tolist()
docs_df[docs_df['docno'].isin(docNoResult)]

جامع قلم


,docno,raw_text,text
1,d2,المحاضرة باللغة العربية لطلاب جامعة القلمون,محاضره لغه عربيه لطلاب جامع قلم
3,d4,نأمل أن تفيد هذا المقرر طلاب جامعة القلومن,نامل تفيد مقرر طلاب جامع قلوم
